In [ ]:
import sys
sys.path.append("/Users/zizhouhuang/Desktop/polyfem-python/build/")
import polyfempy as pf
import json
import torch

torch.set_default_dtype(torch.float64)

In [ ]:
# Differentiable simulator that computes initial derivatives
class Simulate(torch.autograd.Function):

    @staticmethod
    def forward(ctx, solver, body_ids, initial_velocities):
        # Update solver setup
        for bid, vel in zip(body_ids, initial_velocities):
            print(bid, vel)
            solver.set_initial_velocity(bid, vel)
        sys.stdout.flush()
        # Enable caching intermediate variables in the simulation, which will be used for solve_adjoint
        solver.set_cache_level(pf.CacheLevel.Derivatives)
        # Run simulation
        solver.solve()
        # Cache solver for backward gradient propagation
        ctx.solver = solver
        ctx.bids = body_ids
        return torch.tensor(solver.get_solutions())

    @staticmethod
    @torch.autograd.function.once_differentiable
    def backward(ctx, grad_output):
        # solve_adjoint only needs to be called once per solver, independent of number of types of optimization variables
        ctx.solver.solve_adjoint(grad_output.detach().numpy())
        # Compute initial derivatives
        grads = pf.initial_velocity_derivative(ctx.solver)
        flat_grad = torch.zeros((len(ctx.bids), len(grads[ctx.bids[0]])), dtype=float)
        for id, g in grads.items():
            flat_grad[ctx.bids.index(id), :] = torch.tensor(g)
        return None, None, flat_grad

In [ ]:
root = "../data/differentiable/input"
with open(root + "/initial-contact.json", "r") as f:
    config = json.load(f)

config["root_path"] = root + "/initial-contact.json"

# Simulation 1

solver1 = pf.Solver()
solver1.set_settings(json.dumps(config), False)
solver1.set_log_level(2)
solver1.load_mesh_from_settings()

# Simulation 2

config["initial_conditions"]["velocity"][0]["value"] = [3, 0]
solver2 = pf.Solver()
solver2.set_settings(json.dumps(config), False)
solver2.set_log_level(1)
solver2.load_mesh_from_settings()

In [ ]:

# Verify gradient
def loss(param):
    solutions1 = Simulate.apply(solver1, body_ids, param)
    solutions2 = Simulate.apply(solver2, body_ids, param)
    return torch.linalg.norm(solutions1[:, -1]) * torch.linalg.norm(solutions2[:, -1])

torch.set_printoptions(12)

dt = 0.04
solver1.set_cache_level(pf.CacheLevel.Derivatives)
solver1.build_basis()
solver1.assemble()
solver1.init_timestepping(0, dt)
solver2.set_cache_level(pf.CacheLevel.Derivatives)
solver2.build_basis()
solver2.assemble()
solver2.init_timestepping(0, dt)
param = torch.tensor([[5., 0], [0, 0]], requires_grad=True)
body_ids = [1, 3]

theta = torch.randn_like(param)
l = loss(param)
l.backward()
grad = param.grad
t = 1e-6
with torch.no_grad():
    analytic = torch.tensordot(grad, theta)
    f1 = loss(param + theta * t)
    f2 = loss(param - theta * t)
    fd = (f1 - f2) / (2 * t)
    print(f'\ngrad {analytic}, fd {fd} {(f1 - l) / t} {(l - f2) / t}, relative err {abs(analytic - fd) / abs(analytic):.3e}')
    print(f'f(x+dx)={f1}, f(x)={l.detach()}, f(x-dx)={f2}')
    assert(abs(analytic - fd) <= 1e-4 * abs(analytic))